In [5]:
import numpy as np
import pandas as pd
import tushare as ts

from tqdm import tqdm
import warnings
import time

import ast
import jieba
from gensim.models import word2vec
from gensim.models import FastText
from glove import Glove, Corpus

pro = ts.pro_api('52119551ebc32c734a42fd61ccf33344fde3c6195b0aacec58d15140')
ENDDATE = '20190622'

# Import previous data

In [6]:
df_sina = pro.news(src='sina', start_date='20140401', end_date=ENDDATE)
# df_wallstreetcn = pro.news(src='wallstreetcn', start_date='20140401', end_date=ENDDATE)
df_10jqka = pro.news(src='10jqka', start_date='20140401', end_date=ENDDATE)
df_eastmoney = pro.news(src='eastmoney', start_date='20140401', end_date=ENDDATE)
df_yuncaijing = pro.news(src='yuncaijing', start_date='20140401', end_date=ENDDATE)

In [8]:
text_df = [df_sina, df_10jqka, df_eastmoney, df_yuncaijing]

In [9]:
for i, df in enumerate(text_df):
    df['datetime'] = df['datetime'].str.split(' ').apply(lambda x: x[0])
    df[str(i)+'_content'] = df['content']
    df[str(i)+'_title'] = df['title']
    df.drop(columns=["content", "title"], inplace=True)

In [10]:
news_df = df_sina
# news_df = news_df.merge(df_wallstreetcn, on="datetime", how="left")
news_df = news_df.merge(df_10jqka, on="datetime", how="left")
news_df = news_df.merge(df_eastmoney, on="datetime", how="left")
news_df = news_df.merge(df_yuncaijing, on="datetime", how="left")
news_df.head()

MemoryError: 